# Optimize EV Charging Load

**Author:** Eleanor Adachi

**Last updated:** 1/8/2025

**Description:** This is a test of the new EV charging optimization algorithm for all years and multiple combined scenarios for a random sample of 100 circuits, minimizing *annual* net peak when minimizing net peak. Run time with 3 combined scenarios and 3 years is ~1 min 30 sec for `minimize_net_peak`.

In [1]:
# import packages
import datetime as dt
import itertools
import json
import matplotlib.pyplot as plt
import pandas as pd

from optimize_ev_load import *

In [2]:
# INPUTS

# specify optimization weighting; weights should be between 0.0 and 1.0 and add up to 1.0
opt_wt_dict = {
    'minimize_net_peak': 0.0,
    'minimize_grid_need': 1.0
}

# read combined scenarios to include in analysis
with open(r'..\parameters\combined_scenarios.json') as json_file:
    combined_sc_dict = json.load(json_file)
# # TEST
# combined_sc_dict = {
# 	"B": {
# 		"_description": "high scenario for building electrification + more residential scenario for EV charging",
# 		"EVres": 4,
# 		"EVcom": 4, 
# 		"BE": 4
# 	}
# }
# combined_sc_dict = {
# 	"C": {
# 		"_description": "high scenario for building electrification + standard scenario for EV charging",
# 		"EVres": 1,
# 		"EVcom": 1,
# 		"BE": 4
# 	}
# }

# # set up list of years
# # option 1: specify years (see option 2 later)
# # yr_ls = [2050]
# yr_ls = [2040]

# specify limit type
feedlim_type = 'limit_b_md_kw'

In [3]:
# read in addload
addload_all = pd.read_csv(r'..\data\addload.csv')

# TODO: Investigate why getting "inf" grid need for feeder 13921102
# for now, skip 13921102
addload_all = addload_all[ addload_all['feeder_id'] != 13921102 ]

# read in feedlim
feedlim_all = pd.read_csv(r'..\data\feedlim.csv')

# read in feedload
feedload = pd.read_csv(r'..\data\feedload.csv')

# read in feedvre
feedvre = pd.read_csv(r'..\data\feedvre.csv')

In [4]:
# test/100_feeders
test_feeders_df = pd.read_csv(r'..\data\test100\test100_feeders_v3.csv')

# filter to only include feeders in the 100 feeder sample
addload_all = pd.merge(test_feeders_df, addload_all, how='left', on='feeder_id')
feedlim_all = pd.merge(test_feeders_df, feedlim_all, how='left', on='feeder_id')

# INSTEAD of pro-rating based on # feeders, filter feedload and feedvre to only include feeders in the 100 feeder sample and then add up later
feedload = pd.merge(test_feeders_df, feedload, how='left', on='feeder_id')
feedvre = pd.merge(test_feeders_df, feedvre, how='left', on='feeder_id')

In [5]:
# option 2: get all unique years in addload
# WARNING - this will overwrite yr_ls defined previously
ldinc_cols = [x for x in list(addload_all.columns) if x.startswith('ldinc')]
yr_ls = list(set(map(lambda x: int(x.split('_')[2]), ldinc_cols)))
yr_ls.sort()

In [6]:
# initialize newload_all DataFrame
newload_all = pd.DataFrame(columns=['feeder_id', 'month', 'hour', 'mhid'])

# initialize gn_kw_all DataFrame
gn_kw_all = addload_all[['feeder_id']]
gn_kw_all = gn_kw_all.drop_duplicates().reset_index(drop=True)

In [7]:
# filter feedlim
feedlim = feedlim_all[['feeder_id', 'month', 'hour', feedlim_type]]

# rename columns
feedlim = feedlim.rename(columns={feedlim_type: 'limit'})

In [8]:
# initialize np_kw_dict, for full-year optimization
np_kw_dict = {}

# create addload_base_cols
addload_base_cols = ['feeder_id', 'month', 'hour', 'mhid']

# initialize netload_all
netload_all = pd.DataFrame(columns=['month', 'hour', 'mhid'])

In [9]:
# start time
start_dt = dt.datetime.now()

# iterate across years and scenarios
for sc_id in combined_sc_dict.keys():
    print('Combined Scenario: ',sc_id)
    for yr in yr_ls:
        print('Year: ',yr)
        
        # select load-scenario dictionary based on combined scenario ID
        load_sc_dict = combined_sc_dict[sc_id]

        # filter & rename columns
        keep_cols = addload_base_cols.copy() # need to copy to "reset" the list
        rename_col_dict = {}
        load_type_ls = list(filter(lambda x: not(x.startswith('_')), load_sc_dict.keys())) # exclude keys that start with underscore
        for load_type in load_type_ls:
            sc_num = load_sc_dict[load_type]
            load_sc_yr_col = f'ldinc_{load_type}{sc_num}_{yr}_kW'
            keep_cols.append(load_sc_yr_col)
            rename_col_dict[load_sc_yr_col] = load_type
        addload = addload_all[ keep_cols ]
        addload = addload.rename(columns=rename_col_dict)
        
        # get list of unique load types
        other_loads = list(set(load_type_ls) - set(['EVres', 'EVcom']))
        # combine other loads
        addload['other'] = addload[ other_loads ].sum(axis=1)
        # drop other loads
        addload = addload.drop(columns=other_loads)

        # create input_df (merge addload and feedlim; INNER merge to ensure that data is available)
        input_df = pd.merge(addload, feedlim, how='left', on=['feeder_id', 'month', 'hour'])

        # remove feeders with any NaN values
        input_na_df = pd.isna(input_df.set_index('feeder_id')).sum(axis=1)
        remove_feeders = list(set(input_na_df[ input_na_df > 0 ].index))
        n_removed = len(remove_feeders)
        print('Removing %d feeders: ' % n_removed, remove_feeders)
        input_df = input_df[ ~input_df['feeder_id'].isin(remove_feeders) ]

        # remove feeders with any NaN values from feedload and feedvre
        feedload_filt = feedload[ ~feedload['feeder_id'].isin(remove_feeders) ]
        feedvre_filt = feedvre[ ~feedvre['feeder_id'].isin(remove_feeders) ]

        # create base_vre
        baseload = feedload_filt[['month', 'hour', 'mhid', 'l_kW', 'h_kW']].groupby(['month', 'hour', 'mhid']).sum()
        baseload = baseload.reset_index()
        vre_profile = feedvre_filt[['month', 'hour', 'solar_kW', 'wind_kW']].groupby(['month', 'hour']).sum()
        vre_profile = vre_profile.reset_index()
        base_vre = pd.merge(baseload, vre_profile, how='inner', on=['month', 'hour'])
        base_vre = base_vre.rename(columns={'h_kW':'base'})
        base_vre['vre'] = base_vre['solar_kW'] + base_vre['wind_kW']
        base_vre = base_vre[['month', 'hour', 'mhid', 'base', 'vre']]

        # create sys_input_df (merge base_vre and addload sums (system-wide); OUTER merge), for full-year optimization
        sys_addload = addload.pivot_table(values=['EVres', 'EVcom', 'other'], index=['month', 'hour', 'mhid'], aggfunc='sum').reset_index()
        sys_input_df = pd.merge(base_vre, sys_addload, how='outer', on=['month', 'hour', 'mhid'])
        
        # create grid need dataframe
        # NOTE: Positive "grid need" means that loads exceed limits (i.e. upgrades required), negative "grid need" means that there is "headroom"
        gn_kw_df = make_gn_kw_df(input_df, opt=False)

        # get np_kw
        np_kw_df = make_np_kw_df(base_vre, input_df, opt=False)
        np_kw = np_kw_df['np_kW'].max() # for full-year optimization
        
        # count number of circuits
        feednum = len(input_df['feeder_id'].unique())
        print('Number of feeders included in optimization: ', feednum)
        
        # optimize EV charging
        if opt_wt_dict['minimize_net_peak'] == 0.0 and opt_wt_dict['minimize_grid_need']==1.0:
            print('Running grid need-minimizing algorithm...')
            output_df, gn_kw_opt_df = minimize_grid_need(input_df)
            # output_df, gn_kw_opt_df = minimize_grid_need(input_df, EVres_charging_hours=[0,1,2,3,4,5,6,7,19,20,21,22,23], EVcom_charging_hours=range(9,18)) # expanded charging hours
            # get np_kw_opt
            np_kw_opt_df = make_np_kw_df(base_vre, output_df, opt=True)
            np_kw_opt = np_kw_opt_df['np_kW_opt'].max() # for full-year optimization
        elif opt_wt_dict['minimize_net_peak'] == 1.0 and opt_wt_dict['minimize_grid_need']==0.0:
            print('Running net peak-minimizing algorithm...')
            output_df, np_kw_opt = minimize_net_peak_annual(input_df, sys_input_df) # for full-year optimization
            # output_df, np_kw_opt = minimize_net_peak_annual(input_df, sys_input_df, EVres_charging_hours=[0,1,2,3,4,5,6,7,19,20,21,22,23], EVcom_charging_hours=range(9,18)) # expanded charging hours
            # merge in feedlim and make gn_kw_opt_df
            output_df = pd.merge(output_df, feedlim, how='inner', on=['feeder_id', 'month', 'hour']) # NEW
            gn_kw_opt_df = make_gn_kw_df(output_df, opt=True)
        else:
            raise ValueError('Multi-objective optimization not supported yet')

        # save to newload, index by sc_id
        # if newload_all.empty:
        #     newload_all = output_df[['feeder_id', 'month', 'hour', 'mhid', 'EVres_opt', 'EVcom_opt']]
        # else:
        #     newload = output_df[['feeder_id', 'month', 'hour', 'mhid', 'EVres_opt', 'EVcom_opt']]
        #     newload = newload.rename(columns={'EVres_opt': f'ldinc_EVres{sc_id}_{yr}_kW_opt', 'EVcom_opt': f'ldinc_EVcom{sc_id}_{yr}_kW_opt'})
        #     newload_all = pd.merge(newload_all, newload, how='outer', on=['feeder_id', 'month', 'hour', 'mhid'])
        newload = output_df[['feeder_id', 'month', 'hour', 'mhid', 'EVres_opt', 'EVcom_opt']]
        newload = newload.rename(columns={'EVres_opt': f'ldinc_EVres{sc_id}_{yr}_kW_opt', 'EVcom_opt': f'ldinc_EVcom{sc_id}_{yr}_kW_opt'})
        if newload_all.empty:
            newload_all = newload
        else:
            newload_all = pd.merge(newload_all, newload, how='outer', on=['feeder_id', 'month', 'hour', 'mhid'])

        # create netload_df and save to netload_all
        netload_df = make_netload_df(output_df, feedload, feedvre, include_opt=True)
        netload_df = netload_df.rename(columns={'netload': f'netload_{sc_id}_{yr}_MW', 'netload_opt': f'netload_{sc_id}_{yr}_MW_opt'})
        if netload_all.empty:
            netload_all = netload_df.copy()
        else:
            netload_all = pd.merge(netload_all, netload_df, how='outer', on=['month', 'hour', 'mhid'])
        
        # merge gn_kw_df and gn_kw_opt_df, zero out any negative values and change dtype to float
        gn_kw_df = pd.merge(gn_kw_df, gn_kw_opt_df, how='outer', on='feeder_id')
        gn_kw_df['gn_kW'] = gn_kw_df['gn_kW'].clip(lower=0).astype('float')
        gn_kw_df['gn_kW_opt'] = gn_kw_df['gn_kW_opt'].clip(lower=0).astype('float')

        # add gn_kw_df to gn_kw_all
        gn_kw_df = gn_kw_df.rename(columns={'gn_kW': f'gn_{sc_id}_{yr}_kW', 'gn_kW_opt': f'gn_{sc_id}_{yr}_kW_opt'})
        gn_kw_all = pd.merge(gn_kw_all, gn_kw_df, how='outer', on='feeder_id')

        # add to np_kw_dict, for full-year optimization
        np_kw_dict[f'np_{sc_id}_{yr}_kW'] = np_kw
        np_kw_dict[f'np_{sc_id}_{yr}_kW_opt'] = np_kw_opt

# end time
end_dt = dt.datetime.now()

print('Elapsed time: ', end_dt - start_dt)

Combined Scenario:  B
Year:  2030
Removing 3 feeders:  [62021104, 183052113, 254611108]
Number of feeders included in optimization:  65
Running grid need-minimizing algorithm...
Year:  2040
Removing 3 feeders:  [62021104, 183052113, 254611108]
Number of feeders included in optimization:  65
Running grid need-minimizing algorithm...
Year:  2050
Removing 3 feeders:  [62021104, 183052113, 254611108]
Number of feeders included in optimization:  65
Running grid need-minimizing algorithm...
Combined Scenario:  C
Year:  2030
Removing 3 feeders:  [62021104, 183052113, 254611108]
Number of feeders included in optimization:  65
Running grid need-minimizing algorithm...
Year:  2040
Removing 3 feeders:  [62021104, 183052113, 254611108]
Number of feeders included in optimization:  65
Running grid need-minimizing algorithm...
Year:  2050
Removing 3 feeders:  [62021104, 183052113, 254611108]
Number of feeders included in optimization:  65
Running grid need-minimizing algorithm...
Combined Scenario:  

In [10]:
newload_all.tail()

,feeder_id,month,hour,mhid,ldinc_EVresB_2030_kW_opt,ldinc_EVcomB_2030_kW_opt,ldinc_EVresB_2040_kW_opt,ldinc_EVcomB_2040_kW_opt,ldinc_EVresB_2050_kW_opt,ldinc_EVcomB_2050_kW_opt,...,ldinc_EVresC_2040_kW_opt,ldinc_EVcomC_2040_kW_opt,ldinc_EVresC_2050_kW_opt,ldinc_EVcomC_2050_kW_opt,ldinc_EVresD_2030_kW_opt,ldinc_EVcomD_2030_kW_opt,ldinc_EVresD_2040_kW_opt,ldinc_EVcomD_2040_kW_opt,ldinc_EVresD_2050_kW_opt,ldinc_EVcomD_2050_kW_opt
18715,255262105,12,19,284,621.812,90.414,1243.624,180.827,2487.247,361.655,...,897.631,329.400,1795.261,658.801,343.245,205.387,686.489,410.773,1372.978,821.546
18716,255262105,12,20,285,472.149,62.434,944.298,124.867,1888.596,249.734,...,672.611,197.022,1345.223,394.045,258.664,120.367,517.328,240.733,1034.657,481.466
18717,255262105,12,21,286,355.831,44.054,711.661,88.108,1423.322,176.216,...,502.836,131.490,1005.672,262.980,196.254,73.066,392.508,146.131,785.016,292.263
18718,255262105,12,22,287,283.330,25.977,566.661,51.955,1133.322,103.910,...,400.796,68.006,801.591,136.012,161.516,40.912,323.032,81.824,646.065,163.648
18719,255262105,12,23,288,225.437,18.340,450.873,36.680,901.746,73.361,...,318.743,44.782,637.486,89.564,127.956,25.730,255.912,51.461,511.824,102.921


In [11]:
gn_kw_all.tail()

,feeder_id,gn_B_2030_kW,gn_B_2030_kW_opt,gn_B_2040_kW,gn_B_2040_kW_opt,gn_B_2050_kW,gn_B_2050_kW_opt,gn_C_2030_kW,gn_C_2030_kW_opt,gn_C_2040_kW,gn_C_2040_kW_opt,gn_C_2050_kW,gn_C_2050_kW_opt,gn_D_2030_kW,gn_D_2030_kW_opt,gn_D_2040_kW,gn_D_2040_kW_opt,gn_D_2050_kW,gn_D_2050_kW_opt
63,254121102,576.375,88.404,3260.578,2914.121,6547.459,4828.249,182.826,10.609,2760.838,2760.838,4973.260,4521.683,0.000,0.000,2668.290,2668.290,4336.588,4336.588
64,254611108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,254702104,68.213,22.666,3120.294,3029.201,4210.981,4028.795,65.145,32.574,3114.160,3049.017,4198.715,4068.429,63.822,39.254,3111.513,3062.377,4193.421,4095.148
66,255121103,632.262,623.931,2322.563,2257.082,3042.785,2985.758,736.336,736.336,2481.891,2481.891,3435.376,3435.376,808.412,808.412,2626.043,2626.043,3723.679,3723.679
67,255262105,1232.380,1232.380,6898.840,6898.840,11801.714,9948.977,1133.669,1133.669,6701.420,6701.420,9554.137,9554.137,1068.786,1068.786,6571.651,6571.651,9294.599,9294.599


In [12]:
# for full-year optimization
np_kw_all = pd.DataFrame.from_dict(np_kw_dict, orient='index', columns=['kW'])
np_kw_all = np_kw_all.T
np_kw_all

,np_B_2030_kW,np_B_2030_kW_opt,np_B_2040_kW,np_B_2040_kW_opt,np_B_2050_kW,np_B_2050_kW_opt,np_C_2030_kW,np_C_2030_kW_opt,np_C_2040_kW,np_C_2040_kW_opt,np_C_2050_kW,np_C_2050_kW_opt,np_D_2030_kW,np_D_2030_kW_opt,np_D_2040_kW,np_D_2040_kW_opt,np_D_2050_kW,np_D_2050_kW_opt
kW,404642.347123,404642.347123,527173.195123,527173.195123,630852.31507,614927.348123,402253.184123,402253.184123,522394.869123,522394.869123,605370.703123,605370.703123,400537.109123,400537.109123,518962.714123,518962.714123,598506.394123,598506.394123


In [13]:
netload_all.head()

,month,hour,mhid,netload_B_2030_MW,netload_B_2030_MW_opt,netload_B_2040_MW,netload_B_2040_MW_opt,netload_B_2050_MW,netload_B_2050_MW_opt,netload_C_2030_MW,...,netload_C_2040_MW,netload_C_2040_MW_opt,netload_C_2050_MW,netload_C_2050_MW_opt,netload_D_2030_MW,netload_D_2030_MW_opt,netload_D_2040_MW,netload_D_2040_MW_opt,netload_D_2050_MW,netload_D_2050_MW_opt
0,1,0,1,244.997405,219.963060,370.220583,319.021251,545.407878,444.983895,223.193625,...,326.613048,290.864828,458.192797,389.056662,209.365734,196.406312,298.957250,271.579074,402.881222,350.375386
1,1,1,2,232.130544,209.791352,352.751880,307.832071,522.671058,434.196855,210.898098,...,310.286978,278.758583,437.741258,377.132894,197.111663,185.428481,282.714102,258.906122,382.595506,337.470882
2,1,2,3,209.439802,201.443691,314.395350,298.745318,456.581634,425.432159,192.178401,...,279.872559,268.980848,387.536045,367.590457,180.416535,176.582325,256.348813,248.475842,340.488541,326.888569
3,1,3,4,193.139030,197.103129,285.216123,294.026592,403.567356,420.875358,179.011269,...,256.960595,263.933706,347.056288,362.593984,169.093191,172.030456,237.124458,243.161170,307.384017,321.475873
4,1,4,5,188.791375,196.883339,276.452947,293.716804,386.826766,420.628505,175.819685,...,250.509568,263.737706,334.939998,362.568825,166.886731,171.966978,232.643656,243.116874,299.208183,321.816730


In [14]:
# save results
gn_pct = int(round(opt_wt_dict['minimize_grid_need']*100, 0))
np_pct = int(round(opt_wt_dict['minimize_net_peak']*100, 0))

newload_all.to_csv(r'..\results\test100\newload_gn%d_np%d_test100.csv' % (gn_pct, np_pct), index=False)
gn_kw_all.to_csv(r'..\results\test100\gridneed_gn%d_np%d_test100.csv' % (gn_pct, np_pct), index=False)
np_kw_all.to_csv(r'..\results\test100\netpeak_gn%d_np%d_test100.csv' % (gn_pct, np_pct), index=False)
netload_all.to_csv(r'..\results\test100\netload_gn%d_np%d_test100.csv' % (gn_pct, np_pct), index=False)
# # expanded charging hours
# newload_all.to_csv(r'..\results\test100\newload_gn%d_np%d_test100_expchg.csv' % (gn_pct, np_pct), index=False)
# gn_kw_all.to_csv(r'..\results\test100\gridneed_gn%d_np%d_test100_expchg.csv' % (gn_pct, np_pct), index=False)
# np_kw_all.to_csv(r'..\results\test100\netpeak_gn%d_np%d_test100_expchg.csv' % (gn_pct, np_pct), index=False)
# netload_all.to_csv(r'..\results\test100\netload_gn%d_np%d_test100_expchg.csv' % (gn_pct, np_pct), index=False)